#### City & Population

##### - import set up

In [1]:
import os
import requests
import pandas as pd
from sqlalchemy import create_engine
from gans_connection import get_database_connection

# Retrieve the API key from environment variables
from dotenv import load_dotenv
load_dotenv()
city_api_key = os.environ.get('CITY_API_KEY')

##### - create a function do get data from the city API

In [2]:
def get_city_info(city):
    api_url = f'https://api.api-ninjas.com/v1/city?name={city}'
    response = requests.get(api_url, headers={'X-Api-Key': city_api_key})

    try:
        response.status_code == requests.codes.ok
        data = response.json()
        return data
    except Exception as e:
        print(f"Uh-oh, unexpected POPULATION anomaly X.X {e}")

##### - gathers and filters city information from a list of cities

In [3]:
def get_cities_infos(cities):
    cities_table = [get_city_info(city) for city in cities]
    return [city_info for city_info in cities_table if city_info is not None]

##### - create a handler to create the population table

In [4]:
def population_handler(cities_df, engine):
    try:
        # Create population df
        population = cities_df[['name', 'population']].rename(columns={'name': 'city_name'})

        # Fetch cities table
        cities_table = pd.read_sql_table('cities', engine)
        cities = pd.DataFrame(cities_table)

        # lets grag only the col that we need
        cities = cities[['city_id', 'city_name']]

        # create the population table
        # merge to get city_id
        population_df = population.merge(cities, on='city_name', how='left')

        # drop city_name
        population_df = population_df.drop(columns='city_name', axis=1)

        # add year
        population_df['timestamp_population'] = pd.to_datetime('now').year

        # store to population table
        population_df.to_sql('populations', engine, if_exists='append', index=False)

        return population_df
    except Exception as e:
        print(f"Uh-oh, unexpected POPULATION anomaly X.X {e}")


##### - create the main handler for create cities & population tables

In [5]:
def city_pop_handler():
    cities = ["Geneva","Lisbon",]
    try:
        # Retrieve data and flatten the list of lists
        cities_table = get_cities_infos(cities)
        cities_flat = [item for sublist in cities_table for item in sublist]

        # Create DataFrame
        cities_df = pd.DataFrame(cities_flat)

        # Obtain the database engine using the imported function
        engine = get_database_connection()

        # Extract required columns for 'cities' table
        cities_table = cities_df[['name', 'latitude', 'longitude', 'country']].rename(
            columns={'name': 'city_name', 'country': 'country_code'}
        )

        # Store 'cities' table in the database
        cities_table.to_sql('cities', engine, if_exists='append', index=False)
        print(cities_table)
        population_handler(cities_df,engine)
        
    except Exception as e:
        print(f"Uh-oh, unexpected CITY POP anomaly X.X {e}")
        
    return print("Data dancing delightedly :D ")



##### - Let's the magic happen

In [6]:
city_pop_handler()

  city_name  latitude  longitude country_code
0    Geneva   46.2000     6.1500           CH
1    Lisbon   38.7452    -9.1604           PT
Data dancing delightedly :D 
